In [1]:
import scanpy as sc
import patsy as ptsy
import torch
import numpy as np

/home/salvatore.milite/miniconda3/envs/scdeepaa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adata = sc.datasets.ebi_expression_atlas("E-MTAB-8559", filter_boring = True)

sc.pp.filter_genes(adata, min_cells=100)
sc.pp.filter_cells(adata, min_genes=200)

In [3]:
# a bit of metadata cleaning

adata.obs.columns = adata.obs.columns.str.replace("Factor Value[", "")
adata.obs.columns = adata.obs.columns.str.replace("]", "")
# remove duplicated coloumn
adata.obs = adata.obs.drop(columns = "individual")
adata.obs.columns = adata.obs.columns.str.replace("Sample Characteristic[", "")
adata.obs.columns = adata.obs.columns.str.replace("growth condition", "growth_condition")



In [4]:
adata.obs.head()

,age,individual,tumor grading,growth_condition,n_genes
SAMEA6492740-AAACCCACAGTTAGGG,81 year,38b,International Federation of Gynecology and Obs...,OCMI media supplemented with 5% hyclone serum,3117
SAMEA6492740-AAACCCACATGTGTCA,81 year,38b,International Federation of Gynecology and Obs...,OCMI media supplemented with 5% hyclone serum,2973
SAMEA6492740-AAACCCAGTCGCATGC,81 year,38b,International Federation of Gynecology and Obs...,OCMI media supplemented with 5% hyclone serum,3795
SAMEA6492740-AAACCCAGTCTTTCAT,81 year,38b,International Federation of Gynecology and Obs...,OCMI media supplemented with 5% hyclone serum,3706
SAMEA6492740-AAACCCATCCGTGTCT,81 year,38b,International Federation of Gynecology and Obs...,OCMI media supplemented with 5% hyclone serum,2174


In [5]:
X = adata.X

In [6]:
sc_transofrm_v2 = ptsy.dmatrix("~ 1", adata.obs)
covariates = ptsy.dmatrix("~ growth_condition - 1", adata.obs)
random_effects = ptsy.dmatrix("~ individual - 1 ", adata.obs)


In [7]:
import sys
sys.path.append("../src/")

In [12]:
%load_ext autoreload
%autoreload 2

import pydevil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
#UMI = np.array((X.sum(1).squeeze(1) /  X.sum(1).mean())).squeeze()
UMI = np.array(X.sum(1)).squeeze()
UMI.shape

(19880,)

In [15]:
gene_names = adata.var_names

In [16]:
obs_names = adata.obs_names

In [30]:
res  = pydevil.run_SVDE(X.todense(), random_effects, UMI,
                        gene_names = gene_names,
                        cell_names = obs_names, 
                        #group_matrix = random_effects,
                        optimizer = "Adam", lr = 0.005, steps = 500, 
                        full_cov = True, cuda = True)

cuda:0


ELBO: 8.39916  : 100%|██████████| 500/500 [00:31<00:00, 15.95it/s]


In [23]:
contrast = np.array([0,1,-1,0])

In [1]:
marker_genes = pydevil.test_posterior(res, contrast, 0.05)

NameError: name 'pydevil' is not defined

In [ ]:
marker_genes.loc[marker_genes.is_significant]

In [ ]:
sc.pl.dotplot(pbmc, marker_genes_dict, 'cell type', dendrogram=True)


In [31]:
X.todense().shape

(19880, 16288)